# Input Data

For this project, I pulled the last 10 years of bowl games and around 500 data points for each team every year. Note there were around 15k missing datapoints, in those cases we assume the team performed "average" and interpolate a value from the average of all other teams that year.

In [ ]:
import pandas as pd
import glob
import numpy as np
!pip install Unidecode

from joblib import parallel_backend

parallel_backend("loky", n_jobs=30)

# Reading all csv files starting with '20', adding a new column 'year' extracted from the filename, and concatenating them into a single DataFrame
historical_stats = pd.concat(
    [
        pd.read_csv(file).assign(
            year=int(file.split("-")[0])
        )  # Reading each csv file and adding a 'year' column
        for file in glob.glob("20*.csv")  # Finding all csv files starting with '20'
    ]
)

# Reading a csv file named 'games.csv' into a DataFrame
historical_games = pd.read_csv("games.csv")
games_trained_on = historical_games.shape[0] 

# Load in this years bowl games
bowl_games_2023 = pd.read_csv("bowl_games_2023.csv")
num_eval_games = len(bowl_games_2023)

# Get some fun stats
total_nans = historical_stats.isnull().sum().sum()
total_entries = historical_stats.size

# SEED RANDOM
np.random.seed(1)

In [ ]:
fixed_stats = historical_stats.copy()

# Replace missing data points with the average
total_nans = fixed_stats.isnull().sum().sum()
total_entries = fixed_stats.size
fixed_stats.fillna(fixed_stats.mean(numeric_only=True), inplace=True)

# Split win loss
fixed_stats[["Wins", "Losses"]] = fixed_stats["Win-Loss"].str.split(
    "-", expand=True
)

# Bad Columns - These are data points without much reported data
fixed_stats.drop(['Time of Possession', 'Average Time of Possession per Game', 'Win-Loss'], axis=1, inplace=True, errors='ignore')
assert fixed_stats.isnull().sum().sum() == 0, "Nans!"

# Remove duplicates
fixed_stats.drop_duplicates(subset=["Team", "year"], keep="first", inplace=True)
duplicates = fixed_stats[fixed_stats.duplicated(["Team", "year"])]
assert duplicates.shape[0] == 0, "Duplicates found!"


In [ ]:
from unidecode import unidecode

def fix_team_names(df, cols=["0_team", "1_team"]):
    def helper(team_name):
        team_name = team_name.rsplit("(", 1)[0].strip()
        team_name = team_name.replace("St.", "State")
        team_name = ''.join(filter(lambda x: (x.isalpha() or x == " ") , team_name))
        team_name = team_name.lower()
        team_name = unidecode(team_name)
        return team_name
    
    for col in cols:
        df[col] = df[col].apply(helper)

    return df

problem = ""
def verify_team_stats_exist(frame, stats_df, team_cols=["0_team", "1_team"]):
    not_found_teams = []

    for _, row in frame.iterrows():
        year = row["year"]
        teams =  [row[name] for name in team_cols]
        for team in teams:
            if stats_df[
                (stats_df["year"] == year) & (stats_df["Team"] == team)
            ].empty:
                not_found_teams.append((team, year, teams))
                problem = team

    return not_found_teams

def replace_and_save(df, old_string, new_string, filename):
    df.replace(old_string, new_string, inplace=True)
    df.to_csv(filename, index=False)

def delete_team_rows(df, team_name,filename, cols=["0_team", "1_team"]):
    for col in cols:
        df = df[df[col] != team_name]
    df.to_csv(filename, index=False)
    return df

# Normalize all df team names
fixed_stats = fix_team_names(fixed_stats, cols=["Team"])
bowl_games_2023 = fix_team_names(bowl_games_2023)
historical_games = fix_team_names(historical_games)

# Check training data names
error = verify_team_stats_exist(historical_games, fixed_stats)
assert len(error) == 0, f"Not all teams in historical game data found in historical stats {len(error)}" + str(error[:5])

# Check our evaluation data names
error = verify_team_stats_exist(bowl_games_2023, fixed_stats)
assert len(error) == 0, f"Not all teams in bowl game history found in historical stats {len(error)}" + str(error[:5])


In [ ]:
def add_team_stats(games_df, stats_df, is_training_data=True):
    new_df = pd.DataFrame()
    new_df["year"] = games_df["year"]
    new_df["0_team"] = games_df["0_team"]
    new_df["1_team"] = games_df["1_team"]

    if is_training_data: 
        new_df["id_winner"] = games_df["winner_id"]
    else:
        new_df["id_winner"] = None

    # Merge historical stats for team 0
    new_df = pd.merge(
        new_df,
        stats_df.add_prefix("0_"),
        how="inner",
        left_on=["year", "0_team"],
        right_on=["0_year", "0_Team"],
    )

    # Merge historical stats for team 1
    new_df = pd.merge(
        new_df,
        stats_df.add_prefix("1_"),
        how="inner",
        left_on=["year", "1_team"],
        right_on=["1_year", "1_Team"],
    )

    # Remove the columns used to merge
    new_df = new_df.drop(columns=["0_team", "1_team", "0_year", "0_Team", "1_year", "1_Team"])

    return new_df

# Attach stats to training and eval data
training_data = add_team_stats(historical_games, fixed_stats)
training_data_nums = training_data.size
evaluation_data = add_team_stats(bowl_games_2023, fixed_stats, is_training_data=False)

# Model Training

Each model was trained on the whole data set, with 10-fold cross validation to determine its accuracy. 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np

models = {}

from joblib import parallel_backend

parallel_backend("loky", n_jobs=30)


def train_model(pipeline, data=training_data):
    # Extract the features and target variable from the training data
    X_train = data.drop("id_winner", axis=1)
    Y_train = data["id_winner"]

    # Train the model with 5-fold cross validation
    scores = cross_val_score(pipeline, X_train, Y_train, cv=5)

    # Train the model on the entire dataset
    pipeline.fit(X_train, Y_train)

    return pipeline, np.mean(scores)


def eval_model(name, model, data=training_data):
    trained_model, accuracy = train_model(model, data)
    models[name] = {
        "model": trained_model,
        "accuracy": accuracy,
    }

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

# Create a pipeline
pipeline = Pipeline([("scaler", StandardScaler()), ("classifier", GaussianNB())])

eval_model("2022 Model", pipeline)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

# Create a pipeline
pipeline = Pipeline([("scaler", MinMaxScaler()), ("classifier", GaussianNB())])

eval_model("Naive Bayes", pipeline)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# Define the three classifiers to use in the ensemble
clf1 = GaussianNB()
clf2 = MLPClassifier(hidden_layer_sizes=(15,), max_iter=500)
clf3 = RandomForestClassifier(n_estimators=50)

# Combine the classifiers in the ensemble model
ensemble_model = VotingClassifier(
    estimators=[("gnb", clf1), ("mlp", clf2), ("rf", clf3)], voting="soft"
)

eval_model("Ensemble", ensemble_model)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# Create pipelines with different scalers and logistic regression model
pipeline = Pipeline(
    [("scaler", MinMaxScaler()), ("logreg", LogisticRegression(max_iter=1000))]
)

# Fit the pipelines to the training data and print the accuracy for each one
eval_model("MM LogReg", pipeline)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# Create pipelines with different scalers and logistic regression model
pipeline = Pipeline(
    [ ('scalar', MinMaxScaler()), ('pca', PCA(n_components=50)), ("logreg", LogisticRegression(max_iter=1000))]
)

# Fit the pipelines to the training data and print the accuracy for each one
eval_model("PCA LogReg", pipeline)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# Create pipelines with different scalers and logistic regression model
pipeline = Pipeline(
    [("scaler", StandardScaler()), ("logreg", LogisticRegression(C=1, max_iter=1000, penalty='l1', solver='liblinear'))]
)

# Fit the pipelines to the training data and print the accuracy for each one
eval_model("MM LogReg*", pipeline)

/home/hexuser/.cache/pypoetry/virtualenvs/python-kernel-OtKFaj5M-py3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 30.
  warnings.warn(


In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Create a pipeline
pipeline = Pipeline(
    [
        ("variance_threshold", VarianceThreshold()),
        ("scalar", StandardScaler()),
        ("logistic_regression", LogisticRegression(max_iter=1000)),
    ]
)

eval_model("VT LogReg", pipeline)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC())),
  ('classification', RandomForestClassifier())
])

eval_model("FS + RFC", clf)

/home/hexuser/.cache/pypoetry/virtualenvs/python-kernel-OtKFaj5M-py3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Model Confdience

To the left is a plot of all the model's confidences relative to the ranked confidence. We can see that a Naive Bayes w/o any scaling becomes very confident very fast, much moreso than all other models. 

* Note the Y-Axis is grounded at 50% because all of these models cannot predict a winner with less than 50% confidence, bc then it would just pick the other team

In [ ]:
model_accuracies = pd.DataFrame(
    [
        (name, model['accuracy'])
        for name, model in models.items()
    ],
    columns=["Model", "Training Accuracy"],
)

In [ ]:
predictions = dict()

for model_name, model_details in models.items():
    model = model_details['model']
    predictions[model_name] = model.predict_proba(
        evaluation_data.drop(columns=["id_winner"])
    )

output = bowl_games_2023.copy()

for model_name, predict in predictions.items():
    output[model_name + "_predicted_winner"] = predict.argmax(axis=1)
    output[model_name+ "_confidence"] = predict.max(axis=1)
    output[model_name + "_rel_confidence"] = output[model_name + "_confidence"].rank(ascending=True, method="min")
    output[model_name + "_predicted_winner_team"] = output.apply(
            lambda row: row["0_team"] if row[model_name + "_predicted_winner"] == 0 else row["1_team"], axis=1
        )

available_models_df = pd.DataFrame(list(models), columns=["Model"])

# Model Inspection


Change what model you want to inspect its confidence when it comes to ranking

In [ ]:
import json as _hex_json
view_model = _hex_pks.kernel_execution.input_cell.run_dropdown_dynamic(args=_hex_types.DropdownDynamicArgs.from_dict({**_hex_json.loads("{\"dataframe_column\":\"Model\",\"ui_selected_value\":\"MM LogReg*\"}"), **{_hex_json.loads("\"options_variable\""):_hex_kernel.variable_or_none("available_models_df", scope_getter=lambda: globals())}}), app_session_token=_hex_APP_SESSION_TOKEN, python_kernel_init_status=_hex_python_kernel_init_status, hex_timezone=_hex_kernel.variable_or_none("hex_timezone", scope_getter=lambda: globals()), interrupt_event=locals().get("_hex_interrupt_event"))

import json as _hex_json
_hex_pks.kernel_execution.input_cell.filled_dynamic_value(args=_hex_types.FilledDynamicValueArgs.from_dict({**_hex_json.loads("{\"variable_name\":\"available_models_df\",\"dataframe_column\":\"Model\",\"max_size\":10000,\"max_size_in_bytes\":5242880}"), **{_hex_json.loads("\"variable\""):_hex_kernel.variable_or_none("available_models_df", scope_getter=lambda: globals())}}), app_session_token=_hex_APP_SESSION_TOKEN, python_kernel_init_status=_hex_python_kernel_init_status, hex_timezone=_hex_kernel.variable_or_none("hex_timezone", scope_getter=lambda: globals()), interrupt_event=locals().get("_hex_interrupt_event"))

In [ ]:
confidence_df = output[
    [
        "0_team",
        "1_team",
        "{}_predicted_winner".format(view_model),
        "{}_rel_confidence".format(view_model),
        "{}_confidence".format(view_model),
    ]
]
confidence_df.columns = [
    "Team A",
    "Team B",
    "Projected Winner",
    "Ranked Confidence",
    "Confidence %",
]

confidence_df["Projected Winner"] = confidence_df.apply(
            lambda row: row["Team A"] if row["Projected Winner"] == 0 else row["Team B"], axis=1
        )

/tmp/ipykernel_12/3006726547.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confidence_df["Projected Winner"] = confidence_df.apply(


# Summary

Last year a simple Naive Bayes Classifier was used to get lackluster performance, this year we are going with a Logistic Regression. Bring it

In [ ]:
import json as _hex_json
select_which_model_to_save = _hex_pks.kernel_execution.input_cell.run_dropdown_dynamic(args=_hex_types.DropdownDynamicArgs.from_dict({**_hex_json.loads("{\"dataframe_column\":\"Model\",\"ui_selected_value\":\"PCA LogReg\"}"), **{_hex_json.loads("\"options_variable\""):_hex_kernel.variable_or_none("available_models_df", scope_getter=lambda: globals())}}), app_session_token=_hex_APP_SESSION_TOKEN, python_kernel_init_status=_hex_python_kernel_init_status, hex_timezone=_hex_kernel.variable_or_none("hex_timezone", scope_getter=lambda: globals()), interrupt_event=locals().get("_hex_interrupt_event"))

import json as _hex_json
_hex_pks.kernel_execution.input_cell.filled_dynamic_value(args=_hex_types.FilledDynamicValueArgs.from_dict({**_hex_json.loads("{\"variable_name\":\"available_models_df\",\"dataframe_column\":\"Model\",\"max_size\":10000,\"max_size_in_bytes\":5242880}"), **{_hex_json.loads("\"variable\""):_hex_kernel.variable_or_none("available_models_df", scope_getter=lambda: globals())}}), app_session_token=_hex_APP_SESSION_TOKEN, python_kernel_init_status=_hex_python_kernel_init_status, hex_timezone=_hex_kernel.variable_or_none("hex_timezone", scope_getter=lambda: globals()), interrupt_event=locals().get("_hex_interrupt_event"))

In [ ]:
save_model = _hex_json.loads("false")

In [ ]:
import pickle

selected_model = models[select_which_model_to_save]['model']

# Save the model to disk
if save_model:
    filename = select_which_model_to_save + ".pkl"
    pickle.dump(selected_model, open(filename, "wb"))
    print(f"Saved {selected_model}")

# Biggest Takeaways

1. More data => more better, last year we only trained on the last 10 years of bowl games, this year we expanded to the last 10 years of all FCS and FBS games (excluding games with a team not in one of those conferences)
2. Grid Searching parameters => instead of guess n checking hyper parameters, we actually did an exhaustive search 
3. Using ChadGPT => Chad GPT was very helpful for coming up with different model types, namely the ensemble models
4. Using Hex => this site is actually insanely helpful for iterating

In [ ]:
conf_df = pd.DataFrame()

conf_df.insert(0, "Ranked Confidence", range(1, 43))

for model_name, predict in predictions.items():
    temp = pd.DataFrame()
    temp[model_name + " confidence"] = predict.max(axis=1)
    temp["rel_confidence"] = output[model_name + "_confidence"].rank(
        ascending=True, method="min"
    )
    temp["rel_confidence"] = temp["rel_confidence"].astype(int)
    conf_df = pd.merge(conf_df, temp, left_on="Ranked Confidence", right_on="rel_confidence", how="left")
    conf_df = conf_df.drop(columns=["rel_confidence"])

In [ ]:
conf_df

In [ ]:
print(confidence_df.to_markdown())

|    | Team A             | Team B              | Projected Winner   |   Ranked Confidence |   Confidence % |
|---:|:-------------------|:--------------------|:-------------------|--------------------:|---------------:|
|  0 | ga southern        | ohio                | ohio               |                   9 |       0.591825 |
|  1 | howard             | florida am          | florida am         |                  30 |       0.794975 |
|  2 | jacksonville state | louisiana           | jacksonville state |                  36 |       0.875496 |
|  3 | miami oh           | app state           | miami oh           |                  38 |       0.891484 |
|  4 | new mexico         | fresno state        | fresno state       |                  35 |       0.868388 |
|  5 | ucla               | boise state         | ucla               |                  15 |       0.653845 |
|  6 | california         | texas tech          | texas tech         |                   8 |       0.59097  |
|  7 | wes